### Paths

In [1]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'
FINAL_ANNOTS_PATH = WORKSPACE_PATH+'/annotations'


### Creating training and test sets

In [18]:
import glob
import json
import random

In [7]:
annots = []
for name in glob.glob(ANNOTATION_PATH+'/*.json'):
	annots.append(name)

In [19]:
random.shuffle(annots)

In [20]:
selected_annots = annots[:2000]
len(selected_annots)

2000

In [31]:
selected_imgs = []

for annot in selected_annots: 
	parts = annot.split('\\')
	img_name =  ".".join(parts[1].split(".", 2)[:2])
	selected_imgs.append(img_name)



In [34]:
train_imgs = selected_imgs[:1400]
test_imgs = selected_imgs[1400:]

In [35]:
def move_file(src, dst):
	import shutil
	shutil.move(src, dst)

In [36]:
for img in train_imgs:
	all_images_path = IMAGE_PATH + '/allimages'
	train_path = IMAGE_PATH + '/train'
	move_file(all_images_path+'/'+img, train_path+'/'+img)

In [37]:
for img in test_imgs:
	all_images_path = IMAGE_PATH + '/allimages'
	test_path = IMAGE_PATH + '/test'
	move_file(all_images_path+'/'+img, test_path+'/'+img)

In [41]:
train_path = IMAGE_PATH + '/train'
for img in train_imgs:
	move_file(ANNOTATION_PATH+'/'+img+'.json', train_path+'/'+img+'.json')

In [42]:
test_path = IMAGE_PATH + '/test'
for img in test_imgs:
	move_file(ANNOTATION_PATH+'/'+img+'.json', test_path+'/'+img+'.json')

### Label map

In [43]:
labels = [{'name':'face_with_mask', 'id':1}, {'name':'face_no_mask', 'id':2}]
print(labels)

[{'name': 'face_with_mask', 'id': 1}, {'name': 'face_no_mask', 'id': 2}]


In [70]:
with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

### Create TF records

In [51]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {FINAL_ANNOTS_PATH + '/label_map.pbtxt'} -o {FINAL_ANNOTS_PATH + '/train.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record

2022-04-30 00:17:54.025953: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-30 00:17:54.026541: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-04-30 00:18:00.490341: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-30 00:18:00.492284: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublas64_11.dll'; dlerror: cublas64_11.dll not found
2022-04-30 00:18:00.494137: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublasLt64_11.dll'; dlerror: cublasLt64_11.dll not found
2022-04-30 00:18:00.496129: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cu

In [52]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/test'} -l {FINAL_ANNOTS_PATH + '/label_map.pbtxt'} -o {FINAL_ANNOTS_PATH + '/test.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


2022-04-30 00:20:43.404806: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-30 00:20:43.405610: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-04-30 00:20:55.930694: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-30 00:20:55.934214: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublas64_11.dll'; dlerror: cublas64_11.dll not found
2022-04-30 00:20:55.936776: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublasLt64_11.dll'; dlerror: cublasLt64_11.dll not found
2022-04-30 00:20:55.939252: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cu

### Downloading TF Models Pretrained Models from Tensorflow Model Zoo

In [53]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

Cloning into 'models'...
Updating files:   9% (282/3037)
Updating files:  10% (304/3037)
Updating files:  11% (335/3037)
Updating files:  12% (365/3037)
Updating files:  13% (395/3037)
Updating files:  14% (426/3037)
Updating files:  15% (456/3037)
Updating files:  16% (486/3037)
Updating files:  17% (517/3037)
Updating files:  18% (547/3037)
Updating files:  19% (578/3037)
Updating files:  19% (592/3037)
Updating files:  20% (608/3037)
Updating files:  21% (638/3037)
Updating files:  22% (669/3037)
Updating files:  23% (699/3037)
Updating files:  24% (729/3037)
Updating files:  25% (760/3037)
Updating files:  26% (790/3037)
Updating files:  27% (820/3037)
Updating files:  28% (851/3037)
Updating files:  29% (881/3037)
Updating files:  30% (912/3037)
Updating files:  31% (942/3037)
Updating files:  31% (961/3037)
Updating files:  32% (972/3037)
Updating files:  33% (1003/3037)
Updating files:  34% (1033/3037)
Updating files:  35% (1063/3037)
Updating files:  36% (1094/3037)
Updating fi

### Custom model

In [2]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

### Updating Config For Transfer Learning

In [3]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [4]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [5]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [73]:
config

{'model': ssd {
   num_classes: 2
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   match

In [74]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [75]:
pipeline_config.model.ssd.num_classes = 2
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= FINAL_ANNOTS_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [FINAL_ANNOTS_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = FINAL_ANNOTS_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [FINAL_ANNOTS_PATH + '/test.record']


In [76]:
pipeline_config

model {
  ssd {
    num_classes: 2
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999998989515007e-05
          }
        }
        initializer {
          random_normal_initializer {
            mean: 0.0
            stddev: 0.009999999776482582
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.996999979019165
          scale: true
          epsilon: 0.0010000000474974513
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
        additional_layer_depth: 128
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
 

In [77]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

### Model training

In [78]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=5000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=5000


### Loading Train Model From Checkpoint

In [5]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [6]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

### Real-time detection

In [7]:
import cv2 
import numpy as np
from datetime import datetime, timedelta

In [8]:
category_index = label_map_util.create_category_index_from_labelmap(FINAL_ANNOTS_PATH+'/label_map.pbtxt')

In [9]:
category_index

{1: {'id': 1, 'name': 'face_with_mask'}, 2: {'id': 2, 'name': 'face_no_mask'}}

In [10]:
import alert

In [11]:
minimum_gap = timedelta(minutes = 5)

In [12]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

last_alert_time = datetime.now()
first_alert = False
detection_results = []

while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    label_id_offset = 1

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    
    image_np_with_detections = image_np.copy()

    #print(detections['detection_classes']+label_id_offset)
    detected_class_array = detections['detection_classes']+label_id_offset
    vals, counts = np.unique(detected_class_array, return_counts = True)
    index = np.argmax(counts)
    # print('Detected class = ', category_index[vals[index]]['name'])
    
    # print('detection:', detection_results)
    if len(detection_results) == 0:
        detection_results.append(vals[index])
    else:

        if detection_results[-1] == vals[index]:
            detection_results.append(vals[index])
            if len(detection_results) == 10 and detection_results[0] == 2:
                if first_alert or (datetime.now() - last_alert_time > minimum_gap):
                    alert.send_alert()
                    last_alert_time = datetime.now()
                    first_alert = False
                detection_results = []
        else:
            detection_results = []
            
    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=1,
                min_score_thresh=.3,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break


In [6]:
cap.release()


### Multiple people

In [ ]:
# Setup capture
cap = cv2.VideoCapture('walking.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

last_alert_time = datetime.now()
first_alert = False
detection_results = []

while True:
    ret, frame = cap.read()
    image_np = np.array(frame)

    input_tensor = tf.convert_to_tensor(
        np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    label_id_offset = 1

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(
        np.int64)

    image_np_with_detections = image_np.copy()

    #print(detections['detection_classes']+label_id_offset)
    detected_class_array = detections['detection_classes']+label_id_offset
    vals, counts = np.unique(detected_class_array, return_counts=True)
    index = np.argmax(counts)
    # print('Detected class = ', category_index[vals[index]]['name'])

    # print('detection:', detection_results)
    if len(detection_results) == 0:
        detection_results.append(vals[index])
    else:

        if detection_results[-1] == vals[index]:
            detection_results.append(vals[index])
            if len(detection_results) == 10 and detection_results[0] == 2:
                if first_alert or (datetime.now() - last_alert_time > minimum_gap):
                    alert.send_alert()
                    last_alert_time = datetime.now()
                    first_alert = False
                detection_results = []
        else:
            detection_results = []

    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes']+label_id_offset,
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=5,
        min_score_thresh=.45,
        agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(
        image_np_with_detections, (800, 600)))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break
